In [7]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import time
import numpy as np

interval = 10*60 #seconds                                                                                                                
now = 1505573241#int(time.time() - 60*60) * 1000  #epoch seconds                                                                        
past = now - interval

chicago_index = "network_weather-2017*"

es_chicago = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60) # Chicago instance                                                

query_chicago = {
    "query": {
    "bool": {
    "must": [
        { "range": {  "timestamp": { "gte": past,  "lte": now,  "format": "epoch_second"  }  } },
        {"exists" : {"field" : "srcSite"}},
        {"exists" : {"field" : "destSite"}},
    ],
    "should": [
                {"exists" : {"field" : "delay_mean"}},
                {"exists" : {"field" : "packet_loss"}},
            ],
    "minimum_should_match": 1,

    }
    }
}

scroll_chicago = scan(es_chicago, query=query_chicago, index=chicago_index, scroll='5m', timeout='5m', size=10000)

counter = 0
start_time = time.time()
for entry in scroll_chicago:

    if not counter%10000:  print("Processing event number ", counter)
    counter += 1

print("%s entries processes in %i seconds" %(counter, time.time() - start_time))

Processing event number  0
Processing event number  10000
Processing event number  20000
Processing event number  30000
Processing event number  40000
Processing event number  50000
Processing event number  60000
Processing event number  70000
Processing event number  80000
84650 entries processes in 5 seconds
